In [8]:
%load_ext autoreload
%autoreload 2

import modified_convcode as mcc
import turbo
import modified_turbo as mt
from commpy import channelcoding as cc
import numpy as np
from commpy.utilities import dec2bitarray, bitarray2dec
from utils import awgn_corrupt

SEED = 0
def get_test_rng(seed=SEED):
    return np.random.default_rng(seed)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
block_len = 10

test_message_bits = np.array([1, 1, 0, 1, 0, 1, 0, 0, 0, 1])

In [16]:
gen1 = np.array([[1, 1, 1, 1, 1], [1, 0 , 1, 1, 1]])
bias1 = np.array([1, 0])
gen2 = np.array([[1, 1, 1, 1, 1], [1, 1, 1, 1, 0]])
bias2 = np.array([1, 1])

trellis1 = mcc.ModifiedTrellis(gen1, bias1, delay=1)
trellis2 = mcc.ModifiedTrellis(gen2, bias2, delay=1)
interleaver = cc.RandInterlv(block_len, 0)

print("trellis1")
print(f"Trellis is code type {trellis1.code_type} and rate {trellis1.k}/{trellis1.n}")
print(f"Code has delay {trellis1.delay}")
print("Output table")
display(trellis1.output_table)
print("Next State table")
display(trellis1.next_state_table)

print("trellis2")
print(f"Trellis is code type {trellis2.code_type} and rate {trellis2.k}/{trellis2.n}")
print(f"Code has delay {trellis2.delay}")
print("Output table")
display(trellis2.output_table)
print("Next State table")
display(trellis2.next_state_table)

trellis1
Trellis is code type default and rate 1/2
Code has delay 1
Output table


array([[2, 1],
       [1, 2],
       [1, 2],
       [2, 1],
       [1, 2],
       [2, 1],
       [2, 1],
       [1, 2],
       [0, 3],
       [3, 0],
       [3, 0],
       [0, 3],
       [3, 0],
       [0, 3],
       [0, 3],
       [3, 0]])

Next State table


array([[ 0,  8],
       [ 0,  8],
       [ 1,  9],
       [ 1,  9],
       [ 2, 10],
       [ 2, 10],
       [ 3, 11],
       [ 3, 11],
       [ 4, 12],
       [ 4, 12],
       [ 5, 13],
       [ 5, 13],
       [ 6, 14],
       [ 6, 14],
       [ 7, 15],
       [ 7, 15]])

trellis2
Trellis is code type default and rate 1/2
Code has delay 1
Output table


array([[3, 0],
       [1, 2],
       [0, 3],
       [2, 1],
       [0, 3],
       [2, 1],
       [3, 0],
       [1, 2],
       [0, 3],
       [2, 1],
       [3, 0],
       [1, 2],
       [3, 0],
       [1, 2],
       [0, 3],
       [2, 1]])

Next State table


array([[ 0,  8],
       [ 0,  8],
       [ 1,  9],
       [ 1,  9],
       [ 2, 10],
       [ 2, 10],
       [ 3, 11],
       [ 3, 11],
       [ 4, 12],
       [ 4, 12],
       [ 5, 13],
       [ 5, 13],
       [ 6, 14],
       [ 6, 14],
       [ 7, 15],
       [ 7, 15]])

In [17]:
[s1, s2, s3] = mt.turbo_encode(test_message_bits, trellis1, trellis2, interleaver)
print("Input stream")
display(test_message_bits)
print("My stream 1")
display(s1)
print(f"Stream 1 is same as message? {np.all(test_message_bits == s1)}")
print("Stream 2")
display(s2)
print("Interleaved Stream 3")
display(s3)

encoding turbo code
[MCC] Modfied msg bits: [1 0 1 0 1 0 0 0 1 0]
[MCC] Num inbits 10
[MCC] Num outbits 20
[MCC] Modfied msg bits: [0 0 1 1 0 0 1 1 1 0]
[MCC] Num inbits 10
[MCC] Num outbits 20
Input stream


array([1, 1, 0, 1, 0, 1, 0, 0, 0, 1])

My stream 1


array([1, 1, 0, 0, 0, 1, 1, 0, 1, 0])

Stream 1 is same as message? False
Stream 2


array([1, 1, 1, 0, 1, 1, 0, 1, 0, 0])

Interleaved Stream 3


array([1, 1, 0, 1, 1, 1, 1, 1, 0, 0])

In [18]:
sigma = 1

rng = get_test_rng()

s1_r = awgn_corrupt(rng, s1, sigma = sigma)
s2_r = awgn_corrupt(rng, s2, sigma = sigma)
s3_r = awgn_corrupt(rng, s3, sigma = sigma)

display(s1_r)
display(s2_r)
display(s3_r)

array([ 1.12573022,  0.86789514, -0.35957735, -0.89509988, -1.53566937,
        1.36159505,  2.30400005, -0.05291904,  0.29626476, -2.26542147])

array([ 0.37672554,  1.04132598, -1.32503077, -1.21879166, -0.24591095,
        0.26773265, -1.54425898,  0.68369984, -0.58836946,  0.04251337])

array([ 0.87146534,  2.36646347, -1.66519467,  1.35151007,  1.90347018,
        1.0940123 ,  0.25650075,  0.07827462, -1.45772583, -0.77980488])

In [19]:
res = mt.hazzys_turbo_decode(s1_r, s2_r, s3_r, trellis1, trellis2, sigma ** 2, 6, interleaver)
res.astype(int)

array([0, 0, 0, 1, 1, 0, 0, 1, 0, 1])